In [333]:
# À rendre mercredi 12:50 à 13:50

import pandas as pd # pour gérer le df et les tweets
import re # pour supprimer les caractères spéciaux lors du nettoyage du tweet
import ast

df = pd.read_json('versailles_tweets_100.json')

class Tweet:
    def __init__(self, auteur, text, hashtags, mentions):
        self.auteur = auteur
        self.text = re.sub("[^a-zA-z0-9 @&é\"\'(§è!çà)\-#°\^¨$*€¥ôÔùÙ‰%`@#£=≠±+:÷\\/…•.;∞¿?,≤≥><]", "", text, 0)
        # problème ici car le Ï est supprimé par exemple, demander à Élie
        # fait que l'attribut text de l'instance est une version "nettoyé" du text en entrée
        try:
            self.hashtags = [i["tag"] for i in hashtags["hashtags"]]
            # si dict de clé "#" est dans dict "entities", donc si il y a des #
            # alors l'attribut # de l'instance est une liste des #
        except:
            self.hashtags = []
            # sinon l'attribut # de l'instance est une liste vide
        try:
            self.mentions = [i["username"] for i in mentions["mentions"]]
        except:
            self.mentions = []
            # même mécanisme que pour l'attribut # mais cette fois-ci avec les mentions

        def get_sentiment(self):
            pass


def push_to_zone_datterissage(tweet):
    """Envoie le tweet propre dans le fichier zone_d'atterissage.txt."""
    fic = open("zone_d'atterissage.txt", "a")
    fic.write(str(tweet.__dict__)+"\n")
    fic.close()


def reset_zone_datterissage():
    """Supprime tout le text du fichier zone_d'atterissage.txt."""
    open("zone_d'atterissage.txt", "w").close()


def create_new_df(fichier):
    """Remplit le df2 avec les tweets (les lignes) du fichier zone_d'atterissage.txt."""
    global df2
    df2 = pd.DataFrame()
    fic = open(fichier, 'r')
    for ligne in fic.readlines():
        df2 = df2.append(ast.literal_eval(ligne), ignore_index=True)
        # ast.literal_eval() permet de faire passer les lignes de zone_d'atterissage.txt comme
        # des dict et non comme des str
    fic.close()


def get_all_hashtags():
    """Crée une liste de tous les hashtags présents dans la zone d'atterissage."""
    global liste_des_hashtags
    liste_des_hashtags = df2["hashtags"].values.tolist()
    liste_des_hashtags = [hashtags for sous_liste in liste_des_hashtags for hashtags in sous_liste] # applati la liste
    liste_des_hashtags = list(dict.fromkeys(liste_des_hashtags)) # supprime les doublons


def get_all_users():
    """Crée une liste de tous les utilisateurs présents dans la zone d'atterissage."""
    global liste_des_utilisateurs
    liste_des_utilisateurs = df2["auteur"].values.tolist()


def get_all_mentions():
    """Crée une liste de tous les utilisateurs mentionnés présents dans la zone d'atterissage."""
    global liste_des_mentions
    liste_des_mentions = df2["mentions"].values.tolist()
    liste_des_mentions = [mentions for sous_liste in liste_des_mentions for mentions in sous_liste] # applati la liste


def get_posts_per_users():
    """Crée un dictionnaire de tous les posts par utilisateurs."""
    global liste_des_posts_par_utilisateurs
    liste_des_posts_par_utilisateurs = dict()
    for index, lignes in df2.iterrows():
        if lignes['auteur'] not in liste_des_posts_par_utilisateurs.keys():
            liste_des_posts_par_utilisateurs[lignes['auteur']] = lignes
        else:
            liste_des_posts_par_utilisateurs[lignes['auteur']] = lignes['auteur'], lignes


def get_posts_per_hashtags():
    """Crée un dictionnaire de tous les posts par hashtags."""
    get_all_hashtags()
    global liste_des_posts_par_hashtags
    liste_des_posts_par_hashtags = dict()
    for index, lignes in df2.iterrows():
        for hashtag in liste_des_hashtags:
            if hashtag in lignes['hashtags']:
                if hashtag not in liste_des_posts_par_hashtags.keys():
                    liste_des_posts_par_hashtags[hashtag] = lignes
                else:
                    liste_des_posts_par_hashtags[hashtag] = liste_des_posts_par_hashtags[hashtag], lignes


def get_posts_per_mentions():
    """Crée un dictionnaire de tous les posts par mentions."""
    get_all_mentions()
    global liste_des_posts_par_mentions
    liste_des_posts_par_mentions = dict()
    for index, lignes in df2.iterrows():
        for mention in liste_des_mentions:
            if mention in lignes['mentions']:
                if mention not in liste_des_posts_par_mentions.keys():
                    liste_des_posts_par_mentions[mention] = lignes
                else:
                    liste_des_posts_par_mentions[mention] = liste_des_posts_par_mentions[mention], lignes


def get_top_k_hashtags(k):
    """Affiche le top k des hashtags les plus utilisé."""
    get_all_hashtags()
    temp = dict()
    for i in liste_des_hashtags:
        if i not in temp.keys():
            temp[i] = 1
        else:
            temp[i] += 1
    temp = list(map(list, temp.items())) # transforme le dictionnaire temp en liste de listes
    for x in temp:
        x[0], x[1] = x[1], x[0]
    temp.sort(reverse = True)
    try:
        a = temp[k]
        print("Top", k, "des hashtags les plus utilisés :")
        for i in range(k):
            print(temp[i][1], "utilisé", temp[i][0], "fois")
    except:
        print("Il n'y a pas", k, "hashtags différents. Essayer un nombre inférieur.")


def get_top_k_users(k):
    """Affiche le top k des utilisateurs ayant le plus posté (même structure que get_top_k_hashtags)."""
    get_all_users()
    temp = dict()
    for i in liste_des_utilisateurs:
        if i not in temp.keys():
            temp[i] = 1
        else:
            temp[i] += 1
    temp = list(map(list, temp.items()))
    for x in temp:
        x[0], x[1] = x[1], x[0]
    temp.sort(reverse = True)
    try:
        a = temp[k]
        print("Top", k, "des utilisateurs ayant le plus posté :")
        for i in range(k):
            print(temp[i][1], "a posté", temp[i][0], "fois")
    except:
        print("Il n'y a pas", k, "utilisateurs différents. Essayer un nombre inférieur.")


def get_top_k_mentions(k):
    """Affiche le top k des utilisateurs les plus mentionné(e)s."""
    get_all_mentions()
    temp = dict()
    for i in liste_des_mentions:
        if i not in temp.keys():
            temp[i] = 1
        else:
            temp[i] += 1
    temp = list(map(list, temp.items())) # transforme le dictionnaire temp en liste de listes
    for x in temp:
        x[0], x[1] = x[1], x[0]
    temp.sort(reverse = True)
    try:
        a = temp[k]
        print("Top", k, "des utilisateurs les plus mentionné(e)s :")
        for i in range(k):
            print(temp[i][1], "mentionné.e", temp[i][0], "fois")
    except:
        print("Il n'y a pas", k, "utilisateurs mentionné(e)s différent(e)s. Essayer un nombre inférieur.")


def get_nb_posts_per_user(user):
    """Affiche le nombre de post(s) de l'utilisateur."""
    get_posts_per_users()
    print("L'utilisateur", user, "a posté", len(liste_des_posts_par_utilisateurs[user]) % 3, "fois.")
    # % 3 car le dict qui stock les tweets par user stock : user: user, text, #, mentions (donc 4 éléments)


def get_nb_posts_per_hashtag(hashtag):
    """Affiche le nombre de post(s) contenant le hashtag."""
    get_posts_per_hashtags()
    try:
        print("Il y a", len(liste_des_posts_par_hashtags[hashtag]) % 3, "post(s) avec le hashtag", hashtag)
    except:
        print("Il y a", 0, "post(s) avec le hashtag", hashtag, "\b.")


def get_posts_per_user(user):
    """Retourne l'ensemble des tweets de l'utilisateur spécifié."""
    get_posts_per_users()
    try:
        liste_des_posts_par_utilisateurs[user] == True # Test de vérité
        print("L'utilisateur", user, "a posté :")
        for x in liste_des_posts_par_utilisateurs[user]:
            print(x, end=' ')
    except:
        print("L'utilisateur n'a pas été trouvé ou n'a pas posté.")


def get_posts_per_mention(mention):
    """Retourne l'ensemble des tweets contenant la mention spécifiée."""
    get_posts_per_mentions()
    try:
        liste_des_posts_par_mentions[mention] == True
        print("La mention", mention, "est présente dans le/les tweet(s) :")
        for x in liste_des_posts_par_mentions[mention]:
            print(x['text'], end=' ')
    except:
        print("La mention n'a pas été trouvée.")


# MAINLOOP
reset_zone_datterissage()
# Pour toutes les lignes (donc tous les tweets) de df, création d'un objet de la classe Tweet
# pour la ligne puis envoie de l'objet dans la zone d'atterissage
for index, lignes in df.iterrows():
    push_to_zone_datterissage(Tweet(lignes["_id"], lignes["text"], lignes["entities"], lignes["entities"]))
create_new_df("zone_d\'atterissage.txt")

/var/folders/5t/77z6brvj1wqdf_bkh25x0vhc0000gn/T/ipykernel_83525/1099348287.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(ast.literal_eval(ligne), ignore_index=True)
/var/folders/5t/77z6brvj1wqdf_bkh25x0vhc0000gn/T/ipykernel_83525/1099348287.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(ast.literal_eval(ligne), ignore_index=True)
/var/folders/5t/77z6brvj1wqdf_bkh25x0vhc0000gn/T/ipykernel_83525/1099348287.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(ast.literal_eval(ligne), ignore_index=True)
/var/folders/5t/77z6brvj1wqdf_bkh25x0vhc0000gn/T/ipykernel_83525/1099348287.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pa

In [130]:
df
#df.iloc[1]

,_id,public_metrics,id,conversation_id,author_id,text,geo,lang,created_at,entities,context_annotations,attachments
0,1421616335700824064,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421616335700824064,1421616335700824064,1339914264522461184,Goumin des éléphants joueurs la même fatigue m...,{'place_id': '00b8943291443c8c'},fr,2021-07-31 23:38:41+00:00,"{'hashtags': [{'start': 52, 'end': 63, 'tag': ...",NaN,NaN
1,1421599703116943360,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421599703116943360,1421599703116943360,1339914264522461184,@ericbailly24 @maxigr04del mes tontons vous a...,{'place_id': '00b8943291443c8c'},fr,2021-07-31 22:32:35+00:00,"{'mentions': [{'start': 0, 'end': 13, 'usernam...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN
2,1421599163561742336,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",1421599163561742336,1421599163561742336,1339914264522461184,Ah oui le sommeil là sera compliqué. #CIV est...,{'place_id': '00b8943291443c8c'},fr,2021-07-31 22:30:27+00:00,"{'hashtags': [{'start': 37, 'end': 41, 'tag': ...","[{'domain': {'id': '6', 'name': 'Sports Event'...",NaN
3,1421591889095057408,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421591889095057408,1421591889095057408,1339914264522461184,"31 juillet , journée internationale de la femm...",{'place_id': '00b8943291443c8c'},fr,2021-07-31 22:01:32+00:00,"{'hashtags': [{'start': 60, 'end': 65, 'tag': ...",NaN,NaN
4,1421582795294617600,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421582795294617600,1421582795294617600,717025418,Le pedigree 😂😂😂😂 https://t.co/D3Rv7A2BOF,{'place_id': '00b8943291443c8c'},es,2021-07-31 21:25:24+00:00,"{'urls': [{'start': 17, 'end': 40, 'url': 'htt...",NaN,NaN
5,1421581383454052352,"{'retweet_count': 2, 'reply_count': 0, 'like_c...",1421581383454052352,1421256177359466496,992904738516717568,@isabelle170516 @leonna_julie @Steiner2502 Vou...,{'place_id': '011a304d683c2461'},fr,2021-07-31 21:19:48+00:00,"{'mentions': [{'start': 0, 'end': 15, 'usernam...",NaN,NaN
6,1421575939700445184,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421575939700445184,1421256177359466496,992904738516717568,@LynLyna12 @leonna_julie La grande muette cont...,{'place_id': '011a304d683c2461'},fr,2021-07-31 20:58:10+00:00,"{'mentions': [{'start': 0, 'end': 10, 'usernam...",NaN,NaN
7,1421569996858269696,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421569996858269696,1421569996858269696,736523371,Under wsh 😳😳😳,{'place_id': '011a304d683c2461'},en,2021-07-31 20:34:33+00:00,NaN,NaN,NaN
8,1421563798503272448,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",1421563798503272448,1421563798503272448,1471684208,Les bains d'apollon #versailles #nocturne #app...,"{'place_id': '011a304d683c2461', 'coordinates'...",fr,2021-07-31 20:09:55+00:00,"{'urls': [{'start': 76, 'end': 99, 'url': 'htt...",NaN,NaN
9,1421562928176803840,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",1421562928176803840,1421553997811421184,992904738516717568,@leonna_julie Le rdv aujourd'hui aura tenu se...,{'place_id': '011a304d683c2461'},fr,2021-07-31 20:06:28+00:00,"{'mentions': [{'start': 0, 'end': 13, 'usernam...",NaN,{'media_keys': ['3_1421562925999869960']}


In [137]:
df2

,auteur,text,hashtags,mentions
0,1421616335700824064,Goumin des éléphants joueurs la mme fatigue mm...,[twitter225],[]
1,1421599703116943360,@ericbailly24 @maxigr04del mes tontons vous a...,"[SupportriceMazo, domie, CIV]","[ericbailly24, maxigr04del]"
2,1421599163561742336,Ah oui le sommeil là sera compliqué. #CIV est...,[CIV],[]
3,1421591889095057408,"31 juillet , journée internationale de la femm...",[jifa],[]
4,1421582795294617600,Le pedigree https://t.co/D3Rv7A2BOF,[],[]
5,1421581383454052352,@isabelle170516 @leonna_julie @Steiner2502 Vou...,[],"[isabelle170516, leonna_julie, Steiner2502]"
6,1421575939700445184,@LynLyna12 @leonna_julie La grande muette cont...,[],"[LynLyna12, leonna_julie]"
7,1421569996858269696,Under wsh,[],[]
8,1421563798503272448,Les bains d'apollon #versailles #nocturne #app...,"[versailles, nocturne, appollon]",[]
9,1421562928176803840,@leonna_julie Le rdv aujourd'hui aura tenu se...,[],[leonna_julie]


In [350]:
#print(liste_des_posts_par_mentions)
df2


,auteur,text,hashtags,mentions
0,1421616335700824064,Goumin des éléphants joueurs la mme fatigue mm...,[twitter225],[]
1,1421599703116943360,@ericbailly24 @maxigr04del mes tontons vous a...,"[SupportriceMazo, domie, CIV]","[ericbailly24, maxigr04del]"
2,1421599163561742336,Ah oui le sommeil là sera compliqué. #CIV est...,[CIV],[]
3,1421591889095057408,"31 juillet , journée internationale de la femm...",[jifa],[]
4,1421582795294617600,Le pedigree https://t.co/D3Rv7A2BOF,[],[]
5,1421581383454052352,@isabelle170516 @leonna_julie @Steiner2502 Vou...,[],"[isabelle170516, leonna_julie, Steiner2502]"
6,1421575939700445184,@LynLyna12 @leonna_julie La grande muette cont...,[],"[LynLyna12, leonna_julie]"
7,1421569996858269696,Under wsh,[],[]
8,1421563798503272448,Les bains d'apollon #versailles #nocturne #app...,"[versailles, nocturne, appollon]",[]
9,1421562928176803840,@leonna_julie Le rdv aujourd'hui aura tenu se...,[],[leonna_julie]
